In [4]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

import torch
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision

import matplotlib.pyplot as plt
import pandas as pd

from PIL import Image, ImageOps
import os
import shutil
import random

env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


parametri/config:

In [1]:
#TODO: vratiti na staro, promenjeno zbog google colaba
data_path = "data"

RANDOM_STATE = 1219
N_EPOCHS = 50
BATCH_SIZE = 8
LEARNING_RATE = 0.1
WORKERS = 4

dataset klasa:

In [5]:
import torchvision.transforms.functional as TF

class LatexDataset(Dataset):
    """
    Dataset for image-to-LaTeX project.
    Each item is (image_tensor, formula_string)
    """
    def __init__(self, csv_path: str, transform=None):
        super().__init__()
        self.transform = transform
        df = pd.read_csv(csv_path)
        # promenimo kolonu image tako da ima ceo put do fajla
        df['image'] = df.image.map(lambda x: os.path.join('/content/data/formula_images_processed', f'{x}'))
        # TODO: pojasni si sta je tacno walker
        self.walker = df.to_dict('records')

    def __len__(self):
        return len(self.walker)

    def __getitem__(self, idx):
        item = self.walker[idx]

        formula = item['formula']
        image = torchvision.io.read_image(str(item['image']))
        image = TF.rgb_to_grayscale(image, num_output_channels=1)  # (1, H, W)

        return image, formula

In [13]:
import re
import json
from torch import Tensor

class Text():
    def __init__(self):
        self.pad_id = 0
        self.sos_id = 1
        self.eos_id = 2
        
        self.id2word = json.load(open("data/vocab/100k_vocab.json", "r"))
        self.word2id = dict(zip(self.id2word, range(len(self.id2word))))
        self.TOKENIZE_PATTERN = re.compile(
            r"(\\[a-zA-Z]+)|"           # LaTeX commands like \frac, \sqrt
            r"((\\)*[$-/:-?{-~!\"^_`\[\]])|"  # math symbols
            r"(\w)|"                    # single letters/numbers
            r"(\\)"                     # stray backslashes
            )
        self.n_class = len(self.id2word)

    def int2text(self, x: Tensor):
        return " ".join([self.id2word[i] for i in x if i > self.eos_id])

    def text2int(self, formula: str):
        return torch.LongTensor([self.word2id[i] for i in self.tokenize(formula)])

    def tokenize(self, formula: str):
        tokens = re.finditer(self.TOKENIZE_PATTERN, formula)
        tokens = list(map(lambda x: x.group(0), tokens))
        tokens = [x for x in tokens if x is not None and x != ""]
        return tokens

collate:

In [ ]:

rom torchvision import transforms
from torch.nn.utils.rnn import pad_sequence

text = Text()

def collate_fn(batch, text):

    formulas = [text.text2int(str(i[1])) for i in batch]
    formula_len = torch.tensor([len(f) + 1 for f in formulas], dtype=torch.long)
    formulas = pad_sequence(formulas, batch_first=True)

    batch_size = len(batch)
    sos = torch.full((batch_size, 1), text.sos_id, dtype=torch.long)
    eos = torch.full((batch_size, 1), text.eos_id, dtype=torch.long)
    formulas = torch.cat((sos, formulas, eos), dim=-1)


    images = [i[0] for i in batch]
    max_width, max_height = 0, 0
    for img in images:
        c, h, w = img.size()
        max_width = max(max_width, w)
        max_height = max(max_height, h)

    def pad_image(img):
        c, h, w = img.size()
        padding = (0, 0, max_width - w, max_height - h)
        return torchvision.transforms.functional.pad(img, padding, fill=0)

    images = [pad_image(img) for img in images]
    images = torch.stack(images).to(dtype=torch.float)
    # ---------------------------------------------

    return images, formulas, formula_len

### Enkoder
* input je slika s 3 kanala (RGB)
* output feature map ima `enc_dim` kanala
* `nn.MaxPool2d(2, 1)`  -> asimetricni pooling: praktikuje se za slike teksta, jer je tekst vise sirok nego visok

##### Forward pass
input tenzor za forward pass: `x: (bs, c, w, h)`

`bc = batch size`

`c = number of channels`

`w = image width`

`h = image height`

1. enkodovati `x(bc, c_in, w_in, h_in) -> (bc, c_out, w_out, h_out) `
    *  `c_out` je `enc_dim`
    * `w_out` i `h_out` manji od dimenzija inputa (ofc, conv mreza)
2. permutovati -> da bi feature vector bio poslednji
3. flattenovati
* `encoder_out.shape = (bs, sequence_length = w_out * h_out, d = enc_dim)`

In [10]:
class ConvEncoder(nn.Module):
    def __init__(self, encoder_dim: int):
        super().__init__()
        self.feature_encoder = nn.Sequential(
            nn.Conv2d(1, 64, 3, 1, 1),    # Conv 1
            nn.ReLU(),
            nn.MaxPool2d(2, 2),           # downsample

            nn.Conv2d(64, 128, 3, 1, 1),  # Conv 2
            nn.ReLU(),
            nn.MaxPool2d(2, 2),           # downsample

            nn.Conv2d(128, encoder_dim, 3, 1, 1),  # Conv 3
            nn.ReLU(),
        )
        self.encoder_dim = encoder_dim

    def forward(self, x: Tensor):
        """
        x: (bs, c, w, h)
        encoder_out: (bs, seq_len = w*h, feat_dim = d)
        """
        encoder_out = self.feature_encoder(x)        # (bs, c, w, h)
        encoder_out = encoder_out.permute(0, 2, 3, 1) # (bs, w, h, c)
        bs, w, h, d = encoder_out.size()
        encoder_out = encoder_out.view(bs, -1, d)   # flatten spatial dims
        return encoder_out
        '''class ConvEncoder(nn.Module):
    def __init__(self, encoder_dim: int):
        super().__init__()
        self.feature_encoder = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1),
            nn.Conv2d(64, 128, 3, 1),
            nn.Conv2d(128, 256, 3, 1),
            nn.Conv2d(256, 256, 3, 1),
            nn.MaxPool2d(2, 1),
            nn.Conv2d(256, 512, 3, 1),
            nn.MaxPool2d(1, 2),
            nn.Conv2d(512, encoder_dim, 3, 1),
        )
        self.encoder_dim = encoder_dim

    def forward(self, x: Tensor):
        """
            x: (bs, c, w, h)
            encoder_out: (batch_size, seq_len = width*height, feat_dim = d)
        """
        encoder_out = self.feature_encoder(x)  # (bs, c, w, h) ali c i w manji
        encoder_out = encoder_out.permute(0, 2, 3, 1)  # (bs, w, h, c) poslednja dim je feature depth d
        bs, _, _, d = encoder_out.size()
        encoder_out = encoder_out.view(bs, -1, d) # izravnaj (w, h) matricu u niz w*h tako da (bs, w, h, c) -> (bs, w*h, c)
        return encoder_out'''

# Attention 

In [11]:
class Attention(nn.Module):
    def __init__(self, enoder_dim: int = 512, decoder_dim: int = 512, attention_dim: int = 512):
        super().__init__()

        """
        Racunamo kontekst vektor na osnovu sledecih jednacina
        e = tanh((Wₕhₜ₋₁ + bₕ) + (WᵥV + bᵥ))
        αₜ = Softmax(Wₐ·e + bₐ)
        cₜ = ∑ᵢ αₜⁱ vᵢ, where vᵢ ∈ V
        """
        self.decoder_attention = nn.Linear(decoder_dim, attention_dim, bias=False) # W_h * h_{t-1}
        self.encoder_attention = nn.Linear(enoder_dim, attention_dim, bias=False) # W_V * V
        self.attention = nn.Linear(attention_dim, 1, bias=False)      # W_a * attn

        # Softmax će pretvoriti sirove rezultate u raspodelu verovatnoće (težine pažnje).
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, h: Tensor, V: Tensor):
        """
        Izračunaj kontekst vektor tako što pažljivo posmatraš najrelevantnije delove slike.

        Argumenti:
            h: Prethodno skriveno stanje LSTM dekodera. Oblik: (batch_size, decoder_dim)
            V: Mapa karakteristika. Oblik: (batch_size, w * h, encoder_dim)

        Povratna vrednost:
            context (Tensor): Vektor koji iz mapa karakteristike izvlaci relevantne podatke za generisanje sledeceg karaktera.
                            Oblik: (batch_size, decoder_dime)
        """


        attn_1 = self.decoder_attention(h) #(b, decoder_dim) -> (b, attention_dim)
        attn_2 = self.encoder_attention(V) #(b, w*h, enoder_dim) -> (b, w*h, attention_dim)

        attention= self.attention(torch.tanh(attn_1.unsqueeze(1) + attn_2)).squeeze(2)
        # attn_1.unsqueeze(1): (b, 1, attention_dim)
        # attn_2: (b, w*h, attention_dim)
        # tanh(): (b, w*h, attention_dim)
        # attention: (b, w*h, 1) -> squeeze(2) -> (b, w*h)

        alpha = self.softmax(attention)


        context = (alpha.unsqueeze(2) * V).sum(dim=1)
        # alpha.unsqueeze(2): (b, w*h, 1)
        # V: (b, w*h, enoder_dim)
        # product: (b, w*h, enoder_dim)
        # context: (b, enoder_dim)
        return context

# Dekoder

In [ ]:
class Decoder(nn.Module):
    def __init__(self,n_class: int,embedding_dim: int = 80,encoder_dim: int = 512,decoder_dim: int = 512,attention_dim: int = 512,
        num_layers: int = 1,dropout: float = 0.1,bidirectional: bool = False,sos_id: int = 1,eos_id: int = 2):
        super().__init__()

        """
        Implementacija dekodera za Image-to-Latex model.
        Koristi LSTM ćeliju i Luong pažnju da generiše LaTeX simbole korak po korak.
        cₜ = Attention(hₜ₋₁, V)
        eₜ = Embedding(yₜ)
        (oₜ, hₜ) = LSTM(hₜ₋₁, [cₜ, eₜ])
        p(yₜ₊₁ | y₁, ..., yₜ) = Softmax(Wₒ · oₜ + bₒ)

        """

        self.sos_id = sos_id
        self.eos_id = eos_id

        # Embedding layer konvertuje token ID u vektor
        self.embedding = nn.Embedding(n_class, embedding_dim)  # (vocab_size, embedding_dim)

        # Instanca mehanizma pažnje
        self.attention = Attention(encoder_dim, decoder_dim, attention_dim)  # Veličina enkodera -> veličina pažnje

        # Linearni sloj za spajanje embeddinga i konteksta pažnje
        self.concat = nn.Linear(embedding_dim + encoder_dim, decoder_dim)  # (embedding_dim + encoder_dim) -> decoder_dim

        # Prvi LSTM sloj
        self.rnn = nn.LSTM(
            decoder_dim,
            decoder_dim,
            num_layers,
            batch_first=True,
            bidirectional=bidirectional,
        )

        # Dropout za regularizaciju
        self.dropout = nn.Dropout(dropout)

        # Drugi LSTM sloj za dublji model
        self.rnn2 = nn.LSTM(
            decoder_dim,
            decoder_dim,
            num_layers=1,
            batch_first=True,
            bidirectional=bidirectional,
        )

        # Izlazni sloj koji preslikuje u prostor rečnika
        self.out = nn.Linear(decoder_dim, n_class)  # (decoder_dim) -> (n_class)

        # LogSoftmax za stabilnost prilikom računanja gubitka
        self.logsoftmax = nn.LogSoftmax(dim=-1)

        # Inicijalizacija težina
        self.apply(self.init_weights)

    def init_weights(self, layer):
        if isinstance(layer, nn.Embedding):
            nn.init.orthogonal_(layer.weight)
        elif isinstance(layer, nn.LSTM):
            for name, param in self.rnn.named_parameters():
                if name.startswith("weight"):
                    nn.init.orthogonal_(param)

    def forward(self, y, encoder_out=None, hidden_state=None):
        """
        Generiše sledeći token na osnovu trenutnog stanja i izlaza enkodera.

        Argumenti:
            y: Ulazni tokeni. Oblik: (batch_size, target_len)
            encoder_out: Izlaz enkodera (V). Oblik: (batch_size, encoder_dim, w', h')
            hidden_state: Prethodno skriveno stanje (h, c). Oblik: (num_layers * num_directions, batch_size, decoder_dim)

        Povratna vrednost:
            out: Log-verovatnoće za sledeći token. Oblik: (batch_size, 1, n_class)
            hidden_state: Ažurirano skriveno stanje.
        """

        h, c = hidden_state  # (b, decoder_dim), (b, decoder_dim)

        embed = self.embedding(y)  # (b, seq_len, embedding_dim)
        attention_context = self.attention(h, encoder_out)  # (b, encoder_dim)

        rnn_input = torch.cat([embed[:, -1], attention_context], dim=1)  # (b, embedding_dim + encoder_dim)
        rnn_input = self.concat(rnn_input)  # (b, decoder_dim)

        rnn_input = rnn_input.unsqueeze(1)  # (b, 1, decoder_dim)
        hidden_state = (h.unsqueeze(0), c.unsqueeze(0))  # (1, b, decoder_dim), (1, b, decoder_dim)

        out, hidden_state = self.rnn(rnn_input, hidden_state)  # out: (b, 1, decoder_dim)

        out = self.dropout(out)  # (b, 1, decoder_dim)

        out, hidden_state = self.rnn2(out, hidden_state)  # out: (b, 1, decoder_dim)
        out = self.logsoftmax(self.out(out))  # (b, 1, n_class)

        h, c = hidden_state
        return out, (h.squeeze(0), c.squeeze(0))  # Squeeze dimenziju slojeva

# Model

In [16]:
class Image2LatexModel(nn.Module):
    def __init__(self,n_class: int,embedding_dim: int = 80,encoder_dim: int = 512,decoder_dim: int = 512,attention_dim: int = 512,
        num_layers: int = 1,dropout: float = 0.1,bidirectional: bool = False,text: Text = None, beam_width: int = 5, sos_id: int = 1,eos_id: int = 2):
        super().__init__()
        self.encoder = ConvEncoder(encoder_dim=encoder_dim)
        self.decoder = Decoder(n_class=n_class,embedding_dim=embedding_dim,encoder_dim=encoder_dim,decoder_dim=decoder_dim,attention_dim=attention_dim,num_layers=num_layers,dropout=dropout,bidirectional=bidirectional,sos_id=sos_id,eos_id=eos_id)

        self.init_h = nn.Linear(encoder_dim, decoder_dim)
        self.init_c = nn.Linear(encoder_dim, decoder_dim)
        self.n_class = n_class
        self.text = text
        self.beam_width = beam_width
        self.encoder = ConvEncoder(encoder_dim=encoder_dim)
        self.criterion = nn.CrossEntropyLoss()
    
    # TODO: why do we init the hidden state like this and not just 0?
    def init_decoder_hidden_state(self, V: Tensor):
        """
            input V je autput enkodera (bs, w*h, c)
            return (h, c)
        """
        encoder_mean = V.mean(dim=1)
        h = torch.tanh(self.init_h(encoder_mean))
        c = torch.tanh(self.init_c(encoder_mean))
        return h, c
    
    def forward(self, x: Tensor, y: Tensor, y_len: Tensor):
        encoder_out = self.encoder(x)

        hidden_state = self.init_decoder_hidden_state(encoder_out)

        predictions = []

        for t in range(y_len.max().item()):
            dec_input = y[:, t].unsqueeze(1)
            out, hidden_state = self.decoder(dec_input, encoder_out, hidden_state)
            predictions.append(out.squeeze(1))

        predictions = torch.stack(predictions, dim=1)
        return predictions
    
    def decode_beam_search(self, x, max_length=150):
        encoder_out = self.encoder(x)
        hidden_state = self.init_decoder_hidden_state(encoder_out)

        list_candidate = [([self.decoder.sos_id], hidden_state, 0)]
        for t in range(max_length):
            new_candidates = []
            for inp, state, log_prob in list_candidate:
                y = torch.LongTensor([inp[-1]]).view(BATCH_SIZE, -1).to(x.device)
                out, hidden_state = self.decoder(y, encoder_out, state)

                topk = out.topk(self.beam_width)
                for val, idx in zip(topk.values.view(-1), topk.indices.view(-1)):
                    new_inp = inp + [idx.item()]
                    new_candidates.append((new_inp, hidden_state, log_prob + val.item()))

            new_candidates = sorted(new_candidates, key=lambda x: x[2], reverse=True)
            list_candidate = new_candidates[: self.beam_width]

        return list_candidate[0][0]
    
    def decode(self, x, max_length=150):
        predict = self.decode_beam_search(x, max_length)
        return self.text.int2text(predict)
    
    def compute_loss(self, outputs, formulas_out):
        """
        Compute CrossEntropy loss between predictions and targets.
        outputs: (batch, seq_len, vocab_size) logits
        formulas_out: (batch, seq_len) target token IDs
        """
        bs, t, _ = outputs.size()
        return self.criterion(
            outputs.reshape(bs * t, -1),   # flatten predictions
            formulas_out.reshape(-1)       # flatten targets
        )

In [11]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize(128),   # Resize the *shorter side* to 128 pixels, keeps aspect ratio
    transforms.ToTensor()
])

In [19]:
import numpy as np

def exact_match(pred_list, truth_list):
    em_scores = []
    for pred, truth in zip(pred_list, truth_list):
        len_pred = len(pred)
        len_truth = len(truth)
        max_len = max(len_pred, len_truth)

        # Pad both sequences to the same length
        padded_pred = pred + [""] * (max_len - len_pred)
        padded_truth = truth + [""] * (max_len - len_truth)

        # Calculate EM for this single pair
        em = (np.array(padded_pred) == np.array(padded_truth)).all()
        em_scores.append(em)

    # Return the mean EM score for the entire batch
    return torch.tensor(np.mean(em_scores))

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.cuda.amp import autocast, GradScaler
from nltk.translate.bleu_score import corpus_bleu
MAX_LENGTH=150
EFFECTIVE_BATCH_SIZE = 64

def get_device():
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

def bind_gpu(data):
    device = get_device()
    if isinstance(data, (list, tuple)):
        return [bind_gpu(data_elem) for data_elem in data]
    else:
        return data.to(device, non_blocking=True)

# Priprema podataka
text_processor = Text()

#plz ucitaj ih
train_dataset = LatexDataset('/content/data/im2latex_train.csv', transform=transform)
val_dataset = LatexDataset('/content/data/im2latex_validate.csv', transform=transform)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,  # important for training
    num_workers=WORKERS,
    collate_fn=lambda batch: collate_fn(batch, text)
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,  # usually keep validation deterministic
    num_workers=WORKERS,
    collate_fn=lambda batch: collate_fn(batch, text)
)

# Inicijalizacija modela,greske,scheduler,optimizera
device = get_device()

model = Image2LatexModel(
    n_class=text_processor.n_class,
    text=text_processor,
    beam_width=5,
    sos_id=text_processor.sos_id,
    eos_id=text_processor.eos_id
).to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.002, betas=(0.9, 0.98))

total_steps = (len(train_dataset) // EFFECTIVE_BATCH_SIZE) * N_EPOCHS
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.002, total_steps=total_steps)
accumulation_steps = EFFECTIVE_BATCH_SIZE // BATCH_SIZE

scaler = GradScaler()
#treninng
for epoch in range(N_EPOCHS):
    print(f"Epoch {epoch+1}/{N_EPOCHS}")


    model.train()
    train_loss = 0.0

    for batch_idx, batch in enumerate(train_loader):

        images, formulas, formula_len = bind_gpu(batch)

        formulas_in = formulas[:, :-1]
        formulas_out = formulas[:, 1:]

        with autocast():
          outputs = model(images, formulas_in, formula_len)

          loss = criterion(outputs.reshape(-1, outputs.shape[-1]), formulas_out.reshape(-1))
          loss = loss / accumulation_steps

        scaler.scale(loss).backward()
        train_loss += loss.item() * accumulation_steps

        if (batch_idx + 1) % accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            scheduler.step()

        del images, formulas, outputs, loss
        torch.cuda.empty_cache()


    if (batch_idx + 1) % accumulation_steps != 0:
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()

    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch+1} - Average Train Loss: {avg_train_loss:.4f}")

    # model save
    os.makedirs("/content/drive/My Drive/models", exist_ok=True)
    path = f"/content/drive/My Drive/models/model{epoch+1}.pt"
    torch.save(model, path)

    model.eval()
    val_loss = 0.0
    val_bleu = 0.0
    val_em = 0.0

    with torch.no_grad():
        for batch in val_loader:

            images, formulas, formula_len = bind_gpu(batch)
            formulas_in = formulas[:, :-1]
            formulas_out = formulas[:, 1:]

            with autocast():
              outputs = model(images, formulas_in, formula_len)
              loss = criterion(outputs.reshape(-1, outputs.shape[-1]), formulas_out.reshape(-1))
            val_loss += loss.item()

            predicts = []
            truths = []
            for img, formula in zip(images, formulas):
                pred_tokens = model.decode(img.unsqueeze(0), max_length=MAX_LENGTH)
                truth_tokens = formula.tolist()

                predicts.append(pred_tokens)
                truths.append(truth_tokens)

            predict_strings = [text_processor.tokenize(text_processor.int2text(p)) for p in predicts]
            truth_strings = [text_processor.tokenize(text_processor.int2text(t)) for t in truths]
            print(predict_strings)
            print(truth_strings)
            bleu4 = corpus_bleu([[t] for t in truth_strings], predict_strings)
            em = exact_match(predict_strings, truth_strings)

            val_bleu += bleu4
            val_em += em

    avg_val_loss = val_loss / len(val_loader)
    avg_val_bleu = val_bleu / len(val_loader)
    avg_val_em = val_em / len(val_loader)

    print(f"Validation Loss: {avg_val_loss:.4f}, BLEU4: {avg_val_bleu:.4f}, EM: {avg_val_em:.4f}")

print("Training complete!")
torch.save(model.state_dict(), 'image2latex_model.pth')